In [2]:
println!("Hello Rust")

Hello Rust


()

In [3]:
:dep supply = { path = "../" }

In [4]:
:dep chrono = "0.4"
:dep lazy_static = "1.4"
:dep log = "0.4"
:dep env_logger = "0.11.4"
:dep log4rs = "1.2"
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_yaml = "0.9"
:dep anyhow = "1.0"
:dep tempfile = "3.8"
:dep function_name = "0.3.0"
:dep parking_lot = "0.12"

In [5]:
use supply::demand_planner::DemandPlanner;
use supply::demand::Demand;
use supply::sku::SKU;
use supply::specification::Specification;
use supply::flow::Flow;
use supply::simultaneous_flow::SimultaneousFlow;
use supply::operation::{Operation, MaterialFlowVariant, ResourceFlowVariant};
use chrono::NaiveDate;
use std::sync::Arc;
use parking_lot::Mutex;
use supply::reports::plan_exporter::PlanExporter;
use std::fs;
use function_name::named;
use supply::resource::Resource;
use supply::resource_flow::ResourceFlow;
use supply::operation::OperationVariant;

use supply::logger_config;
use log::LevelFilter;


In [6]:
fn create_date(year: i32, month: u32, day: u32) -> NaiveDate {
    NaiveDate::from_ymd_opt(year, month, day).unwrap()
}

In [7]:
fn create_laptop_supply_chain() -> (
    // DC1 Products
    Arc<Mutex<SKU>>, // Laptop A DC1
    Arc<Mutex<SKU>>, // Laptop B DC1
    // Plant 1 Products
    Arc<Mutex<SKU>>, // Laptop A Plant1
    Arc<Mutex<SKU>>, // Laptop B Plant1
    Arc<Mutex<SKU>>, // Disk
    Arc<Mutex<SKU>>, // CPU
    Arc<Mutex<SKU>>, // Memory 8GB
    Arc<Mutex<SKU>>, // Memory 16GB
    // Resource
    Arc<Mutex<Resource>>, // Assembly Resource
    // Operations
    Arc<Mutex<Operation>>, // Laptop A Assembly
    Arc<Mutex<Operation>>, // Laptop B Assembly
    Arc<Mutex<Operation>>, // Laptop A Transport
    Arc<Mutex<Operation>>  // Laptop B Transport
) {
    // Create SKUs
    let laptop_a_dc1 = SKU::from_name("Laptop_A@DC1");
    let laptop_b_dc1 = SKU::from_name("Laptop_B@DC1");
    let laptop_a_plant1 = SKU::from_name("Laptop_A@Plant1");
    let laptop_b_plant1 = SKU::from_name("Laptop_B@Plant1");
    let disk = SKU::from_name("Disk@Plant1");
    let cpu = SKU::from_name("CPU@Plant1");
    let memory_8gb = SKU::from_name("Memory_8GB@Plant1");
    let memory_16gb = SKU::from_name("Memory_16GB@Plant1");
    
    // Create assembly resource
    let assembly_resource = Resource::from_name("Assemble_Laptop@Plant1");
    let assembly_resource_flow = ResourceFlow::new(1.0, assembly_resource.clone());

    // Create flows for Laptop A assembly
    let laptop_a_output = Flow::new(false, 1.0, laptop_a_plant1.clone());
    let laptop_a_components = SimultaneousFlow::new(vec![
        Flow::new(true, 1.0, disk.clone()),
        Flow::new(true, 1.0, cpu.clone()),
        Flow::new(true, 2.0, memory_8gb.clone()),
    ]);

    // Create flows for Laptop B assembly
    let laptop_b_output = Flow::new(false, 1.0, laptop_b_plant1.clone());
    let laptop_b_components = SimultaneousFlow::new(vec![
        Flow::new(true, 1.0, disk.clone()),
        Flow::new(true, 1.0, cpu.clone()),
        Flow::new(true, 1.0, memory_16gb.clone()),
    ]);

    // Create assembly operations
    let laptop_a_assembly = Operation::new(
        "Make_Laptop_A@Plant1".to_string(),
        1, // lead time
        1, // min batch
        1, // batch multiple
        MaterialFlowVariant::Single(laptop_a_output),
        MaterialFlowVariant::Simultaneous(laptop_a_components),
        ResourceFlowVariant::SingleResource(assembly_resource_flow.clone())
    );

    let laptop_b_assembly = Operation::new(
        "Make_Laptop_B@Plant1".to_string(),
        2, // lead time
        1, // min batch
        1, // batch multiple
        MaterialFlowVariant::Single(laptop_b_output),
        MaterialFlowVariant::Simultaneous(laptop_b_components),
        ResourceFlowVariant::SingleResource(assembly_resource_flow)
    );

    // Create transport operations
    let laptop_a_transport = Operation::new(
        "Move_Laptop_A-to-DC1".to_string(),
        1, // lead time
        1, // min batch
        1, // batch multiple
        MaterialFlowVariant::Single(Flow::new(false, 1.0, laptop_a_dc1.clone())),
        MaterialFlowVariant::Single(Flow::new(true, 1.0, laptop_a_plant1.clone())),
        ResourceFlowVariant::None
    );

    let laptop_b_transport = Operation::new(
        "Move_Laptop_B-to-DC1".to_string(),
        1, // lead time
        1, // min batch
        1, // batch multiple
        MaterialFlowVariant::Single(Flow::new(false, 1.0, laptop_b_dc1.clone())),
        MaterialFlowVariant::Single(Flow::new(true, 1.0, laptop_b_plant1.clone())),
        ResourceFlowVariant::None
    );

    // Set operations on SKUs
    laptop_a_dc1.lock().set_top_producing_operation(OperationVariant::Basic(laptop_a_transport.clone()));
    laptop_b_dc1.lock().set_top_producing_operation(OperationVariant::Basic(laptop_b_transport.clone()));
    laptop_a_plant1.lock().set_top_producing_operation(OperationVariant::Basic(laptop_a_assembly.clone()));
    laptop_b_plant1.lock().set_top_producing_operation(OperationVariant::Basic(laptop_b_assembly.clone()));


    (
        laptop_a_dc1, laptop_b_dc1,
        laptop_a_plant1, laptop_b_plant1,
        disk, cpu, memory_8gb, memory_16gb,
        assembly_resource,
        laptop_a_assembly, laptop_b_assembly,
        laptop_a_transport, laptop_b_transport
    )
}

In [8]:
#[named]
fn test_laptop_planning() {
    /*let path_to_test_settings_yml = "./tests/config/settings_trace_selected_demands.yml";
    if let Err(e) = supply::logger_config::configure_logger_from_path(path_to_test_settings_yml) {
        eprintln!("Failed to configure logger: {}", e);
    }*/
    let _ = logger_config::set_log_level(LevelFilter::Debug);
    
    let (
        laptop_a_dc1, laptop_b_dc1,
        _laptop_a_plant1, _laptop_b_plant1,
        disk, cpu, memory_8gb, memory_16gb,
        assembly_resource,
        _laptop_a_assembly, _laptop_b_assembly,
        _laptop_a_transport, _laptop_b_transport
    ) = create_laptop_supply_chain();

    // Set assembly resource capacity
    let mut assembly_resource_ref = assembly_resource.lock();
    for day in 15..=31 {
        assembly_resource_ref.set_capacity(
            create_date(2024, 1, day),
            100.0 // Daily assembly capacity
        );
    }
    drop(assembly_resource_ref);

    // Add component inventory
    let jan_15 = create_date(2024, 1, 15);
    disk.lock().add_inventory(jan_15, 1000.0);
    cpu.lock().add_inventory(jan_15, 250.0);
    memory_8gb.lock().add_inventory(jan_15, 500.0);
    memory_16gb.lock().add_inventory(jan_15, 500.0);

    // Create demands
    let jan_31 = create_date(2024, 1, 31);
    let demand_a = Demand::new("D1".to_string(), 200.0, jan_31, 1, laptop_a_dc1.clone());
    let demand_b = Demand::new("D2".to_string(), 150.0, jan_31, 2, laptop_b_dc1.clone());

    // Plan demands
    let dp = DemandPlanner::new();
    let mut specification = Specification::new(2, 2);
    //let mut specification = Specification::new_from_settings(2, path_to_test_settings_yml);

    //let result = dp.plan_demand_list(vec![demand_a.clone(), demand_b.clone()], &mut specification);
    let result = dp.plan_demand_list(vec![demand_a.clone(), demand_b.clone()], &mut specification);
    
    assert!(result.is_ok());

    // Export and verify results
    let test_name = function_name!().trim_start_matches("test_");
    PlanExporter::export_all(test_name).unwrap();
    PlanExporter::compare_all(test_name).unwrap();
    fs::remove_dir_all(PlanExporter::get_results_base_dir_parent(test_name)).unwrap();
}


In [9]:
use std::fs;
use std::fs;
use std::io::BufReader;
use std::io::BufRead;

In [10]:
use std::fs::File;
use std::io::{BufReader, BufRead};

fn show_inventory() {
    let test_name = "jupyter";
    let ouput_file = "inventory_profiles.csv";
    let result_dir:String =  format!("{}/{}","..",PlanExporter::get_results_base_dir(test_name));  
    println!("{}",result_dir);
    let test_file = format!("{}/{}", result_dir, ouput_file);
    println!("{}", test_file);
    
    // Read the exported file in the results folder.
    let expect_file = test_file.clone();
    let file = File::open(expect_file).unwrap();
    let reader = BufReader::new(file);
    // Iterate through each line and print in a table-like format
    for line in reader.lines() {
        let line = line.unwrap();
        let fields: Vec<&str> = line.split(',').collect();
        if fields.len() == 4 {
            println!("{:<20} {:<20} {:<12} {:<15}",fields[0],fields[1],fields[2],fields[3]);
        }
    }
}

In [11]:
show_inventory()

../tests/data/run_result/jupyter/expects
../tests/data/run_result/jupyter/expects/inventory_profiles.csv
# SKU Name           Date                 Quantity     Total Quantity 
CPU@Plant1           2024-01-15           250.00       250.00         
CPU@Plant1           2024-01-27           -50.00       200.00         
CPU@Plant1           2024-01-28           -100.00      100.00         
CPU@Plant1           2024-01-29           -100.00      0.00           
Disk@Plant1          2024-01-15           1000.00      1000.00        
Disk@Plant1          2024-01-27           -50.00       950.00         
Disk@Plant1          2024-01-28           -100.00      850.00         
Disk@Plant1          2024-01-29           -100.00      750.00         
Laptop_A@DC1         2024-01-31           0.00         0.00           
Laptop_A@Plant1      2024-01-29           100.00       100.00         
Laptop_A@Plant1      2024-01-30           -100.00      0.00           
Laptop_B@DC1         2024-01-31           0

()

In [12]:
use std::fs::File;
use std::io::{BufReader, BufRead};

fn show_plans(count: i32) {
    let test_name = "jupyter";
    let ouput_file = "operation_plans.csv";
    let result_dir = PlanExporter::get_results_base_dir(test_name);  
    let test_file = format!("{}/{}", result_dir, ouput_file);
    
    // Read the exported file in the results folder.
    let expect_file = test_file.clone();
    let file = File::open(expect_file).unwrap();
    let reader = BufReader::new(file);
    // Iterate through each line and print in a table-like format
    for line in reader.lines() {
        let line = line.unwrap();
        //println!("{}",line);
        let fields: Vec<&str> = line.split(',').collect();
        if fields.len() as i32  == count && count == 4 {
            println!("{:<30} {:<20} {:<12} {:<15}", fields[0],fields[1],fields[2],fields[3]);
        }

        if fields.len() as i32  == count && count == 5 {
            println!("{:<30} {:<20} {:<12} {:<15} {:<15}", fields[0],fields[1],fields[2],fields[3],fields[4]);
        }

        
    }
}

In [13]:
show_plans(4)

thread '<unnamed>' panicked at src/lib.rs:40:40:
called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::result::unwrap_failed
   3: ctx::show_plans
   4: std::panic::catch_unwind
   5: _run_user_code_12
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [14]:
show_plans(5)

thread '<unnamed>' panicked at src/lib.rs:40:40:
called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::result::unwrap_failed
   3: <unknown>
   4: <unknown>
   5: <unknown>
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
